In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import_save = '/content/drive/MyDrive/rdva2024/data/'

import pandas as pd

df_inference = pd.read_csv("/content/drive/MyDrive/rdva2024/data/train_xgb_equal_weights.csv")[0:20000]

In [ ]:
print(df_inference.columns)

Index(['ID', 'parent_asin', 'rating', 'full_text', 'as_image', 'helpful_vote',
       'as_helpful_vote', 'verified_purchase', 'main_category',
       'average_rating', 'rating_number', 'price', 'categories_grp'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>


In [ ]:
def text_and_label(dataset):
    text = []
    label = []

    # Itération sur les lignes du DataFrame avec iterrows()
    for _, row in dataset.iterrows():
        text.append(row["full_text"])  # Accès à la colonne 'full_text'
        label.append(row["rating"])  # Accès à la colonne 'rating'

    # Création du dictionnaire
    return {
        'text': text,
        'label': label
    }

# Exemple d'utilisation
dataset_inference_format = text_and_label(df_inference)

In [ ]:
%pip install torch
%pip install transformers
%pip install huggingface_hub
%pip install peft
%pip install datasets
%pip install -U bitsandbytes transformers
%pip install sklearn
%pip install evaluate
%pip install wandb
%pip install accelerate
%pip install bitsandbytes
%pip install trl
%pip install numpy
%pip install datasets
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
import os
import torch
import huggingface_hub
import sklearn
import numpy as np

from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from torch import nn
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [ ]:
model_dir = '/content/drive/MyDrive/rdva2024/model_llama32_3b_output_EqWtd_2024_12_15_07_02'

id2label = {
    0: 1, 1: 2, 2: 3, 3: 4, 4: 5
}

label2id = {
    1: 0, 2: 1, 3: 2, 4: 3, 5: 4
}

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

llama_classif = AutoModelForSequenceClassification.from_pretrained(
    model_dir, num_labels=5, id2label=id2label, label2id=label2id, quantization_config=quantization_config)

llama_tokenizer = AutoTokenizer.from_pretrained(model_dir)

id2label = {
    0: 1, 1: 2, 2: 3, 3: 4, 4: 5
}

label2id = {
    1: 0, 2: 1, 3: 2, 4: 3, 5: 4
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/885 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
inference_encoding_text = llama_tokenizer(dataset_inference_format["text"], padding=True, truncation=True)

inference_dataset = Dataset.from_dict({"input_ids": inference_encoding_text["input_ids"],
                                   "attention_mask": inference_encoding_text["attention_mask"],
                                   "label": dataset_inference_format["label"]})

inference_dataset = inference_dataset.map(lambda e: {
    'input_ids': torch.tensor(e['input_ids']),
    'attention_mask': torch.tensor(e['attention_mask']),
    'label': torch.tensor(label2id[e['label']])  #
})

dataset = DatasetDict({
    'inference': inference_dataset
})

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
import torch.nn.functional as F

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").long()
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token

llama_classif.config.pad_token_id = llama_tokenizer.pad_token_id

collate_fn = DataCollatorWithPadding(tokenizer=llama_tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    report_to='none'
)

# Création du Trainer
trainer = CustomTrainer(
    model=llama_classif,
    args=training_args,
    tokenizer=llama_tokenizer,
    data_collator=collate_fn
)

<ipython-input-10-9981516e5c40>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [ ]:
# Inference
predictions = trainer.predict(dataset['inference'])
logits = predictions.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
predicted_classes = probs.argmax(axis=-1) + 1
weights = torch.arange(probs.shape[-1]).numpy()
weighted_predictions = (probs * weights).sum(axis=-1) + 1

In [ ]:
df_preds = df_inference

In [ ]:
df_preds['predicted_llm_rating'] = predicted_classes
df_preds['weighted_prediction'] = weighted_predictions

for class_idx in range(probs.shape[1]):
    df_preds[f"prob_rating_{class_idx + 1}"] = probs[:, class_idx]


In [ ]:
print(df_preds.iloc[0])
print(len(df_preds))

ID                                                                9545181
parent_asin                                                    B00IAVDOS6
rating                                                                2.0
full_text               Title : Probably the biggest piece of s**t I'v...
as_image                                                                0
helpful_vote                                                            0
as_helpful_vote                                                         0
verified_purchase                                                   False
main_category                                                 Video Games
average_rating                                                        4.4
rating_number                                                      4831.0
price                                                               36.97
categories_grp                                                Accessories
predicted_llm_rating                  

In [ ]:
df_preds.to_csv(import_save + 'CCO_0_20000_' + 'wtdllm_inference_wtdtrain_customloss.csv', index=False)

In [ ]:
from google.colab import runtime
runtime.unassign()